<a href="https://colab.research.google.com/github/shashigharti/fd-gender-bias-tools-exploration/blob/main/Gender_Bias_Tools_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Dataset used

Placement Data
https://www.kaggle.com/code/benroshan/you-re-hired-analysis-on-campus-recruitment-data/data


**Here we calculate distance between dataset first and apply clustering technique DBSCAN (Density based clustering technique)**

Good part of DBSCAN is, it is robust to outliers. 
Parameters for DBSCAN is epsilon and minPoints

- Gower Distance
- Clustering using DBSCAN(Density based)

References: https://towardsdatascience.com/clustering-on-numerical-and-categorical-features-6e0ebcf1cbad

##Example1 - Using Gower Distance and DBSCAN Clustering to dataset1

In [1]:
!pip install gower

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
from sklearn.cluster import DBSCAN
import gower

In [3]:
# Creating a dictionary with the data
dictionary = {"age": [22, 25, 30, 38, 42, 47, 55, 62, 61, 90], 
              "gender": ["M", "M", "F", "F", "F", "M", "M", "M", "M", "M"], 
              "civil_status": ["SINGLE", "SINGLE", "SINGLE", "MARRIED", "MARRIED", "SINGLE", "MARRIED", "DIVORCED", "MARRIED", "DIVORCED"], 
              "salary": [18000, 23000, 27000, 32000, 34000, 20000, 40000, 42000, 25000, 70000], 
              "has_children": [False, False, False, True, True, False, False, False, False, True], 
              "purchaser_type": ["LOW_PURCHASER", "LOW_PURCHASER", "LOW_PURCHASER", "HEAVY_PURCHASER", "HEAVY_PURCHASER", "LOW_PURCHASER", "MEDIUM_PURCHASER", "MEDIUM_PURCHASER", "MEDIUM_PURCHASER", "LOW_PURCHASER"]}

# Creating a Pandas DataFrame from the dictionary
dataframe = pd.DataFrame.from_dict(dictionary)
dataframe

,age,gender,civil_status,salary,has_children,purchaser_type
0,22,M,SINGLE,18000,False,LOW_PURCHASER
1,25,M,SINGLE,23000,False,LOW_PURCHASER
2,30,F,SINGLE,27000,False,LOW_PURCHASER
3,38,F,MARRIED,32000,True,HEAVY_PURCHASER
4,42,F,MARRIED,34000,True,HEAVY_PURCHASER
5,47,M,SINGLE,20000,False,LOW_PURCHASER
6,55,M,MARRIED,40000,False,MEDIUM_PURCHASER
7,62,M,DIVORCED,42000,False,MEDIUM_PURCHASER
8,61,M,MARRIED,25000,False,MEDIUM_PURCHASER
9,90,M,DIVORCED,70000,True,LOW_PURCHASER


In [4]:
distance_matrix = gower.gower_matrix(dataframe)
print(distance_matrix.shape)
distance_matrix

(10, 10)


array([[0.        , 0.02337858, 0.21512066, 0.7507542 , 0.7669683 ,
        0.06768477, 0.48472852, 0.50829566, 0.45135745, 0.6666667 ],
       [0.02337858, 0.        , 0.19174208, 0.72737557, 0.74358976,
        0.06353696, 0.46134993, 0.48491704, 0.42797887, 0.6432881 ],
       [0.21512066, 0.19174208, 0.        , 0.5356335 , 0.55184764,
        0.23076923, 0.60294116, 0.6265083 , 0.58239067, 0.7848793 ],
       [0.7507542 , 0.72737557, 0.5356335 , 0.        , 0.01621418,
        0.72718704, 0.5673077 , 0.7575415 , 0.5788084 , 0.7492459 ],
       [0.7669683 , 0.74358976, 0.55184764, 0.01621418, 0.        ,
        0.7237934 , 0.5510935 , 0.7413273 , 0.5754148 , 0.7330317 ],
       [0.06768477, 0.06353696, 0.23076923, 0.72718704, 0.7237934 ,
        0.        , 0.41704375, 0.44061086, 0.3836727 , 0.5989819 ],
       [0.48472852, 0.46134993, 0.60294116, 0.5673077 , 0.5510935 ,
        0.41704375, 0.        , 0.19023378, 0.06278281, 0.6819382 ],
       [0.50829566, 0.48491704, 0.6265083

In [5]:
# Configuring the parameters of the clustering algorithm
dbscan_cluster = DBSCAN(eps=0.3, 
                        min_samples=2, 
                        metric="precomputed")

# Fitting the clustering algorithm
clustering = dbscan_cluster.fit(distance_matrix)

# Adding the results to a new column in the dataframe
dataframe["cluster"] = clustering.labels_

dataframe.sort_values(["cluster", "gender","age"], inplace = True)
dataframe

,age,gender,civil_status,salary,has_children,purchaser_type,cluster
9,90,M,DIVORCED,70000,True,LOW_PURCHASER,-1
2,30,F,SINGLE,27000,False,LOW_PURCHASER,0
0,22,M,SINGLE,18000,False,LOW_PURCHASER,0
1,25,M,SINGLE,23000,False,LOW_PURCHASER,0
5,47,M,SINGLE,20000,False,LOW_PURCHASER,0
3,38,F,MARRIED,32000,True,HEAVY_PURCHASER,1
4,42,F,MARRIED,34000,True,HEAVY_PURCHASER,1
6,55,M,MARRIED,40000,False,MEDIUM_PURCHASER,2
8,61,M,MARRIED,25000,False,MEDIUM_PURCHASER,2
7,62,M,DIVORCED,42000,False,MEDIUM_PURCHASER,2


#Result:

Here we see have grouped dataset in 4 clusters. Now we can apply simple techniques to identify the equal representation of all genders.

**Example: How many males and females rows are there in each group?**

In [6]:
dataframe.groupby(['cluster', 'gender']).size().reset_index(name='counts')

,cluster,gender,counts
0,-1,M,1
1,0,F,1
2,0,M,3
3,1,F,2
4,2,M,3


We see some groups have more males than females and some have more females than male. This would be a sign of bias in the dataset

##Example2- Using Gower Distance and DBSCAN Clustering to placement Dataset

###Apply same clustering technique to the placement Dataset

**Description of the fields**

**Sl_no** Serial Number

**Gender** Male/Female

**ssc_p** Secondary Education

**ssc_b** Board of Education

**hsc_b** Board of Education

**hsc_s** Specialization in Higher Secondary Education

**degree_p** Degree Percentage

**degree_t** Under Graduation Degree

**Workex** Work experience

In [7]:
# Reading data
dataframe = pd.read_csv('Placement_Data_Full_Class.csv')
dataframe['salary'] = dataframe['salary'].fillna(0)
dataframe

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,0.0
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0
211,212,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0
212,213,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0
213,214,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0


**Group the data rows based on gender and degree**

We will first calculate the distance of each rows with others and use that distance matrix to cluster the whole dataset

In [8]:
df_n = dataframe[['gender','degree_t']]
df_n

,gender,degree_t
0,M,Sci&Tech
1,M,Sci&Tech
2,M,Comm&Mgmt
3,M,Sci&Tech
4,M,Comm&Mgmt
...,...,...
210,M,Comm&Mgmt
211,M,Sci&Tech
212,M,Comm&Mgmt
213,F,Comm&Mgmt


In [9]:
# Calculate the distance matrix for subset of data
distance_matrix = gower.gower_matrix(df_n)
print("Distance matrix", distance_matrix.shape)
print(distance_matrix[:,10])

# Configuring the parameters of the clustering algorithm
dbscan_cluster = DBSCAN(eps=0.3, 
                        min_samples=5, 
                        metric="precomputed")

# Fitting the clustering algorithm
clustering = dbscan_cluster.fit(distance_matrix)

# Adding the results to a new column in the dataframe
dataframe["cluster"] = clustering.labels_
dataframe

Distance matrix (215, 215)
[0.5 0.5 0.  0.5 0.  0.5 0.5 0.5 0.  0.  0.  0.  0.5 0.5 0.  0.5 0.  0.5
 0.5 0.  0.  0.5 1.  1.  0.5 0.5 0.  0.  0.  0.  0.5 1.  0.5 0.5 0.5 0.5
 0.  1.  0.5 0.5 0.5 0.5 0.5 0.  0.5 1.  0.5 0.  0.  1.  0.5 0.  0.5 0.5
 0.5 0.  0.  0.  0.  0.5 0.  0.  1.  0.  0.  0.  0.  0.  1.  0.5 0.5 0.
 0.  0.  0.  0.5 0.5 0.5 0.5 1.  0.5 0.  0.  0.5 0.5 0.5 0.  0.5 0.5 1.
 0.5 0.  0.5 0.  0.  0.  0.5 0.5 0.5 0.5 0.5 0.  0.5 0.5 0.  0.5 0.5 0.
 0.  0.5 1.  0.5 0.  0.5 0.  0.5 0.  0.5 0.5 0.  0.  1.  0.5 0.  0.5 0.5
 1.  0.5 0.5 0.  0.  1.  0.  0.  0.5 0.5 0.5 0.  1.  0.  0.  0.  0.5 0.
 0.  0.5 0.  0.  1.  0.  0.5 0.  1.  0.5 0.  0.  0.5 0.  0.5 0.  0.5 0.
 0.  0.5 0.5 0.5 0.  0.5 0.5 0.5 0.5 0.  0.  1.  0.5 0.5 0.5 0.5 0.5 1.
 0.  0.5 0.5 0.  0.5 1.  0.5 0.5 0.  0.5 0.5 0.  0.  1.  0.  0.  0.5 1.
 1.  0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0.5 0.  0.  0.5 0.  0.5 0. ]


,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary,cluster
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0,0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0,0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0,1
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,0.0,0
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0,1
211,212,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0,0
212,213,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0,1
213,214,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0,2


In [10]:
dataframe.groupby(['cluster', 'gender', 'degree_t']).size().reset_index(name='counts')

,cluster,gender,degree_t,counts
0,0,M,Sci&Tech,42
1,1,M,Comm&Mgmt,92
2,2,F,Comm&Mgmt,53
3,3,F,Sci&Tech,17
4,4,M,Others,5
5,5,F,Others,6


#Result:

Here we see the clustering is not giving any meaning information but again we can apply the simple grouping technique to identify hidden bias in the dataset.

**Example:Is dataset balanced for each gender in all undergraduate degrees ?**

dataframe.groupby(['cluster', 'gender', 'degree_t']).size().reset_index(name='counts')

**42 males in Science and tech and only 17 female in sci and tech**

##Example3 - Applying fairlens to placement dataset

In [11]:
!pip install fairlens

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import fairlens as fl
fscorer = fl.FairnessScorer(dataframe, "status", ["gender"])
fscorer.sensitive_attrs

['gender']

In [13]:
fscorer.demographic_report()

Sensitive Attributes: ['gender']

Empty DataFrame
Columns: [Group, Distance, Proportion, Counts, P-Value]
Index: []

Weighted Mean Statistical Distance: nan


/usr/local/lib/python3.7/dist-packages/fairlens/scorer.py:260: RuntimeWarning: invalid value encountered in double_scalars
  return (df_dist["Distance"].abs() * df_dist["Counts"]).sum() / df_dist["Counts"].sum()


##Example4 - Applying fairlens to synthetic dataset

In [14]:
# Creating a dictionary with the data
dictionary = {"age": [22, 25, 30, 38, 42, 47, 55, 62, 61, 90], 
              "gender": ["M", "M", "F", "F", "F", "M", "M", "M", "M", "M"], 
              "civil_status": ["SINGLE", "SINGLE", "SINGLE", "MARRIED", "MARRIED", "SINGLE", "MARRIED", "DIVORCED", "MARRIED", "DIVORCED"], 
              "salary": [18000, 23000, 27000, 32000, 34000, 20000, 40000, 42000, 25000, 70000], 
              "has_children": [False, False, False, True, True, False, False, False, False, True], 
              "purchaser_type": ["LOW_PURCHASER", "LOW_PURCHASER", "LOW_PURCHASER", "HEAVY_PURCHASER", "HEAVY_PURCHASER", "LOW_PURCHASER", "MEDIUM_PURCHASER", "MEDIUM_PURCHASER", "MEDIUM_PURCHASER", "LOW_PURCHASER"]}

# Creating a Pandas DataFrame from the dictionary
dataframe = pd.DataFrame.from_dict(dictionary)
dataframe

,age,gender,civil_status,salary,has_children,purchaser_type
0,22,M,SINGLE,18000,False,LOW_PURCHASER
1,25,M,SINGLE,23000,False,LOW_PURCHASER
2,30,F,SINGLE,27000,False,LOW_PURCHASER
3,38,F,MARRIED,32000,True,HEAVY_PURCHASER
4,42,F,MARRIED,34000,True,HEAVY_PURCHASER
5,47,M,SINGLE,20000,False,LOW_PURCHASER
6,55,M,MARRIED,40000,False,MEDIUM_PURCHASER
7,62,M,DIVORCED,42000,False,MEDIUM_PURCHASER
8,61,M,MARRIED,25000,False,MEDIUM_PURCHASER
9,90,M,DIVORCED,70000,True,LOW_PURCHASER


In [15]:
fscorer = fl.FairnessScorer(dataframe, "purchaser_type", ["gender"])
fscorer.sensitive_attrs

['gender']

In [16]:
fscorer.demographic_report()

Sensitive Attributes: ['gender']

Empty DataFrame
Columns: [Group, Distance, Proportion, Counts, P-Value]
Index: []

Weighted Mean Statistical Distance: nan


/usr/local/lib/python3.7/dist-packages/fairlens/scorer.py:260: RuntimeWarning: invalid value encountered in double_scalars
  return (df_dist["Distance"].abs() * df_dist["Counts"]).sum() / df_dist["Counts"].sum()


## Example5 Gender biased text indentifier

In [17]:
!pip3 install genderdecoder3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import genderdecoder3

###Long Text (Taken from OKFN JObs-communication lead)

In [19]:
job_description = """Lead the Foundation communications strategy. This includes setting an overarching strategic communications plan that advances programmatic goals and reflects the Foundation’s values as well as leading operational and tactical work to implement the strategies. 
Oversee communications operations. This includes annual planning and budgeting and ensuring all work is done well and meets the needs of internal and external stakeholders.
Serve as an internal consultant to the CEO and other Foundation staff to develop and refine key messages, themes, presentations that effectively advance the Foundation’s goals and vision. Explore and identify channels, forums and venues for sharing learning and insights.
Provide organisational leadership. This includes working with staff across the foundation to identify and resolve cross-organizational issues and providing leadership, counsel and support for effective internal and external communications."""


In [20]:
output = genderdecoder3.assess(job_description)
output

{'result': 'masculine-coded',
 'explanation': 'This job ad uses more words that are stereotypically masculine than words that are stereotypically feminine. It risks putting women off applying, but will probably encourage men to apply.',
 'masculine_coded_words': ['Lead', 'leading', 'leadership', 'leadership'],
 'feminine_coded_words': ['sharing', 'support']}

In [21]:
print("OUTPUT:", output['result'])
print(output['explanation'])
print("Masculine Coded Words",output['masculine_coded_words'])
print("Feminine Coded Words",output['feminine_coded_words'])

OUTPUT: masculine-coded
This job ad uses more words that are stereotypically masculine than words that are stereotypically feminine. It risks putting women off applying, but will probably encourage men to apply.
Masculine Coded Words ['Lead', 'leading', 'leadership', 'leadership']
Feminine Coded Words ['sharing', 'support']


###Single words

**User Story** [copied from slack by Evgeny]:

You are in Arctic (as our Switzerland friends) and you're writing an Excel file with instructions for data collection during the mission. You open it in Frictionless Application, click check for gender bias you see that it's 20% biased and you need to tweak the instructions

In [22]:
!pip install tabulate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
from tabulate import tabulate

In [24]:
candidates = {"characteristics": ["leader,competitive,expert","principles,lead,strong,self-confident","Individual,Determined,Outspoken"], "title":["programmer", "designer", "teamlead"]}
print(tabulate(candidates, headers="keys"))

characteristics                        title
-------------------------------------  ----------
leader,competitive,expert              programmer
principles,lead,strong,self-confident  designer
Individual,Determined,Outspoken        teamlead


In [25]:
masculine_coded = 0
total_rows = 3
for row in candidates["characteristics"]: 
  output = genderdecoder3.assess(row)
  if output['result'] == 'strongly masculine-coded':
    masculine_coded += 1

print("Is this data mascuiline-coded??")
if masculine_coded > total_rows/2:
  print("Yes it is masculine coded") 
else:
  print("No")
print("Rows with masculine_coded words:", masculine_coded)

Is this data mascuiline-coded??
Yes it is masculine coded
Rows with masculine_coded words: 3


## Using gender neutral

In [26]:
candidates = {"characteristics": ["exceptional,passionate,expert","results-oriented,enthusiastic,skilled,self-confident","experienced,fair,excellent"], "title":["programmer", "designer", "teamlead"]}
masculine_coded = 0
total_rows = 3
for row in candidates["characteristics"]: 
  output = genderdecoder3.assess(row)
  if output['result'] == 'strongly masculine-coded':
    masculine_coded += 1

print("Is this data mascuiline-coded??")
if masculine_coded > total_rows/2:
  print("Yes it is masculine coded")
else:
  print("No")

print("Rows with masculine_coded words:", masculine_coded)

Is this data mascuiline-coded??
No
Rows with masculine_coded words: 0


#Result

We can use this library just to identify masculine words or feminine words. It will work for single text as well.